# 🚀 Cohere Embed v4 + OpenSearch Serverless 멀티모달 임베딩 솔루션

## 📋 개요
이 노트북은 **Amazon Bedrock의 Cohere Embed v4**와 **OpenSearch Serverless**를 사용하여 텍스트와 이미지를 결합한 고성능 멀티모달 검색 시스템을 구현합니다.

## 🎯 주요 기능
- **멀티모달 임베딩**: 텍스트와 이미지를 1536차원 벡터로 통합
- **실시간 벡터 검색**: HNSW 알고리즘 기반 고성능 검색
- **유연한 검색 모드**: 텍스트 전용, 이미지 전용, 멀티모달 검색 지원
- **확장 가능한 아키텍처**: OpenSearch Serverless로 자동 스케일링

## 🏗️ 아키텍처
```
텍스트 + 이미지 → Cohere Embed v4 → 1536D 벡터 → OpenSearch (HNSW) → 검색 결과
```

## 🔧 1. 환경 설정 및 라이브러리 설치

필요한 라이브러리를 설치하고 AWS 클라이언트를 초기화합니다.

### 주요 구성 요소:
- **boto3**: AWS SDK
- **opensearch-py**: OpenSearch 클라이언트
- **Pillow**: 이미지 처리
- **requests-aws4auth**: AWS 인증

In [ ]:
# 라이브러리 설치 및 임포트
!pip install boto3>=1.34.0 opensearch-py>=2.4.0 requests-aws4auth>=1.2.3 pandas>=2.0.0 Pillow>=10.0.0 python-dotenv>=1.0.0

import boto3
import json
import base64
import pandas as pd
from PIL import Image
import io
import os
from datetime import datetime
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 설정
REGION = 'us-east-1'
INDEX_NAME = 'fashion-items-1'

# AWS 클라이언트 생성
bedrock_client = boto3.client('bedrock-runtime', region_name=REGION)
opensearch_client = boto3.client('opensearchserverless', region_name=REGION)
session = boto3.Session()
credentials = session.get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, REGION, 'aoss', session_token=credentials.token)

print("✅ 환경 설정 완료")

## 🔍 2. OpenSearch Serverless 컬렉션 설정

기존 OpenSearch 컬렉션을 찾거나 .env 파일에서 엔드포인트를 로드합니다.

### 설정 방법:
1. **자동 검색**: 활성 상태의 OpenSearch 컬렉션 자동 탐지
2. **수동 설정**: `.env` 파일에 `OPENSEARCH_ENDPOINT` 설정

In [ ]:
# .env에서 엔드포인트 읽기
EXISTING_ENDPOINT = os.getenv('OPENSEARCH_ENDPOINT')

def get_or_create_collection():
    if EXISTING_ENDPOINT:
        print(f"✅ .env에서 엔드포인트 로드: {EXISTING_ENDPOINT[:30]}...")
        return EXISTING_ENDPOINT
    
    try:
        print("🔍 활성 OpenSearch 컬렉션 검색 중...")
        collections = opensearch_client.list_collections()
        for collection in collections["collectionSummaries"]:
            if collection["status"] == "ACTIVE":
                endpoint = f"https://{collection["id"]}.{REGION}.aoss.amazonaws.com"
                print(f"✅ 활성 컬렉션 발견: {collection["name"]} -> {endpoint[:30]}...")
                return endpoint
        
        print("❌ 활성 컬렉션을 찾을 수 없습니다.")
        print("💡 .env 파일에 OPENSEARCH_ENDPOINT를 설정하거나 OpenSearch 컬렉션을 생성해주세요.")
        return None
    except Exception as e:
        print(f"❌ 컬렉션 확인 실패: {e}")
        print("💡 .env 파일에 OPENSEARCH_ENDPOINT를 설정해주세요.")
        return None

ENDPOINT = get_or_create_collection()
if not ENDPOINT:
    raise Exception("OpenSearch 엔드포인트를 찾을 수 없습니다. .env 파일을 확인하거나 컬렉션을 생성해주세요.")

print(f"🎯 사용할 엔드포인트: {ENDPOINT[:30]}...")

## 🔐 3. 데이터 액세스 권한 설정

OpenSearch Serverless에 대한 데이터 액세스 정책을 설정합니다.

### 권한 범위:
- **인덱스 권한**: 생성, 삭제, 업데이트, 읽기, 쓰기
- **컬렉션 권한**: 컬렉션 아이템 생성

In [ ]:
# 데이터 액세스 권한 설정
def setup_data_access_policy():
    try:
        # 현재 사용자 ARN 가져오기
        sts = boto3.client('sts')
        identity = sts.get_caller_identity()
        user_arn = identity['Arn']
        
        # 데이터 액세스 정책
        policy = [
            {
                "Rules": [
                    {
                        "ResourceType": "index",
                        "Resource": ["index/*/*"],
                        "Permission": [
                            "aoss:CreateIndex", "aoss:DeleteIndex", "aoss:UpdateIndex",
                            "aoss:DescribeIndex", "aoss:ReadDocument", "aoss:WriteDocument"
                        ]
                    },
                    {
                        "ResourceType": "collection",
                        "Resource": ["collection/*"],
                        "Permission": ["aoss:CreateCollectionItems"]
                    }
                ],
                "Principal": [user_arn]
            }
        ]
        
        opensearch_client.create_access_policy(
            name='multimodal-data-access-policy',
            type='data',
            policy=json.dumps(policy)
        )
        print(f"✅ 데이터 액세스 정책 생성 완료: {user_arn}")
    except Exception as e:
        if 'ConflictException' in str(e):
            print("ℹ️ 데이터 액세스 정책이 이미 존재합니다")
        else:
            print(f"⚠️ 정책 설정 실패: {e}")

setup_data_access_policy()

## 🏗️ 4. HNSW 벡터 인덱스 생성

고성능 벡터 검색을 위한 HNSW (Hierarchical Navigable Small World) 인덱스를 생성합니다.

### 인덱스 설정:
- **차원**: 1536 (Cohere Embed v4)
- **알고리즘**: HNSW
- **유사도**: 코사인 유사도
- **엔진**: nmslib (고성능)

### HNSW 장점:
- ⚡ **빠른 검색**: O(log N) 시간 복잡도
- 📈 **확장성**: 대용량 데이터 처리
- 🎯 **정확성**: 높은 검색 품질

In [ ]:
# OpenSearch 클라이언트 및 인덱스 생성
client = OpenSearch(
    hosts=[{'host': ENDPOINT.replace('https://', ''), 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300,  # 전역 타임아웃 5분
    max_retries=3,
    retry_on_timeout=True
)

# 1536 차원 인덱스 매핑
mapping = {
    "settings": {"index": {"knn": True}},
    "mappings": {
        "properties": {
            "item_id": {"type": "keyword"},
            "title": {"type": "text"},
            "description": {"type": "text"},
            "category": {"type": "keyword"},
            "brand": {"type": "keyword"},
            "price": {"type": "float"},
            "color": {"type": "keyword"},
            "multimodal_embedding": {
                "type": "knn_vector",
                "dimension": 1536,
                "method": {"name": "hnsw", "space_type": "cosinesimil", "engine": "nmslib"}
            }
        }
    }
}

if client.indices.exists(index=INDEX_NAME, request_timeout=300):
    client.indices.delete(index=INDEX_NAME, request_timeout=300)
client.indices.create(index=INDEX_NAME, body=mapping, request_timeout=300)
print("1536 차원 인덱스 생성 완료")

## 📦 5. 샘플 데이터 생성

테스트용 패션 아이템 데이터와 이미지를 생성합니다.

### 데이터 구조:
- **메타데이터**: 제목, 설명, 카테고리, 브랜드, 가격, 색상
- **이미지**: 색상별 400x400 샘플 이미지

In [ ]:
# 샘플 데이터 생성
os.makedirs('data/images', exist_ok=True)

sample_items = [
    {'item_id': 'item_001', 'title': '블루 데님 재킷', 'description': '캐주얼한 스타일의 클래식 블루 데님 재킷', 'category': '아우터', 'brand': '스타일코', 'price': 89.99, 'color': '블루'},
    {'item_id': 'item_002', 'title': '블랙 이브닝 드레스', 'description': '우아한 블랙 드레스로 특별한 행사에 적합', 'category': '드레스', 'brand': '엘레간스', 'price': 159.99, 'color': '블랙'},
    {'item_id': 'item_003', 'title': '화이트 스니커즈', 'description': '편안하고 스타일리시한 화이트 운동화', 'category': '신발', 'brand': '컴포트스텝', 'price': 79.99, 'color': '화이트'},
    {'item_id': 'item_004', 'title': '레드 울 스웨터', 'description': '따뜻한 겨울용 빨간 울 스웨터', 'category': '상의', 'brand': '웜웨어', 'price': 69.99, 'color': '레드'},
    {'item_id': 'item_005', 'title': '브라운 가죽 핸드백', 'description': '프리미엄 브라운 가죽 핸드백', 'category': '액세서리', 'brand': '레더크래프트', 'price': 129.99, 'color': '브라운'}
]

# 샘플 이미지 생성
colors = {'블루': (70, 130, 180), '블랙': (0, 0, 0), '화이트': (255, 255, 255), '레드': (220, 20, 60), '브라운': (139, 69, 19)}
for item in sample_items:
    color = colors[item['color']]
    img = Image.new('RGB', (400, 400), color)
    img.save(f"data/images/{item['item_id']}.jpg")

print(f"샘플 데이터 생성 완료: {len(sample_items)}개 아이템")

## 🧠 6. 멀티모달 임베딩 함수 정의

Cohere Embed v4를 사용하여 텍스트와 이미지를 결합한 임베딩을 생성합니다.

### 지원 모드:
- **텍스트 전용**: 텍스트만으로 임베딩 생성
- **멀티모달**: 텍스트 + 이미지 결합 임베딩

### 임베딩 특징:
- **차원**: 1536
- **타입**: float
- **용도**: search_document (인덱싱용)

In [ ]:
# 임베딩 생성 함수
def encode_image(image_path):
    with Image.open(image_path) as img:
        if img.mode != 'RGB':
            img = img.convert('RGB')
        buffer = io.BytesIO()
        img.save(buffer, format='JPEG')
        return base64.b64encode(buffer.getvalue()).decode('utf-8')

def generate_embedding(text, image_base64=None):
    if image_base64:
        payload = {
            "input_type": "search_document",
            "embedding_types": ["float"],
            "inputs": [{
                "content": [
                    {"type": "text", "text": text},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}}
                ]
            }]
        }
    else:
        payload = {
            "input_type": "search_document",
            "embedding_types": ["float"],
            "texts": [text]
        }
    
    response = bedrock_client.invoke_model(
        modelId='cohere.embed-v4:0',
        body=json.dumps(payload),
        contentType='application/json'
    )
    
    result = json.loads(response['body'].read())
    if isinstance(result['embeddings'], list):
        return result['embeddings'][0]["float"]
    else:
        return result['embeddings']['float'][0]
        

print("임베딩 함수 정의 완료")

## 🚀 7. 임베딩 생성 및 OpenSearch 업로드

각 아이템에 대해 멀티모달 임베딩을 생성하고 OpenSearch에 인덱싱합니다.

### 처리 과정:
1. **텍스트 구성**: 메타데이터를 구조화된 텍스트로 변환
2. **이미지 인코딩**: Base64로 이미지 인코딩
3. **임베딩 생성**: Cohere Embed v4로 1536차원 벡터 생성
4. **인덱싱**: OpenSearch에 문서 저장

⚠️ **참고**: API 제한을 고려하여 요청 간 2초 대기

In [ ]:
import time
# 임베딩 생성 및 업로드
processed_items = []

for item in sample_items:
    print(f"처리 중: {item['title']}")
    
    # 텍스트 생성
    text = f"Title: {item['title']}\nDescription: {item['description']}\nCategory: {item['category']}\nBrand: {item['brand']}\nColor: {item['color']}\nPrice: ${item['price']}"
    
    # 이미지 인코딩
    image_path = f"data/images/{item['item_id']}.jpg"
    image_base64 = encode_image(image_path)
    
    # 멀티모달 임베딩 생성
    embedding = generate_embedding(text, image_base64)
    time.sleep(2)  # 2초 대기
    
    # 문서 생성
    doc = {
        **item,
        'multimodal_embedding': embedding,
        'created_at': datetime.now().isoformat()
    }
    
    # OpenSearch에 업로드 (타임아웃 5분)
    response = client.index(index=INDEX_NAME, body=doc, request_timeout=300)
    processed_items.append(doc)
    print(f"완료: {item['item_id']} - {response['_id']}")

print(f"\n모든 아이템 처리 완료: {len(processed_items)}개")

## ⚡ 7-2. 다중 입력 처리 (Multi-Input Processing)

여러 아이템을 한 번의 API 호출로 처리하여 성능을 향상시킵니다.

### 📝 용어 설명:
- **다중 입력 처리**: Cohere API의 `inputs` 배열에 여러 항목을 포함하여 한 번에 처리
- **Bedrock Batch Inference**: AWS가 제공하는 S3 기반 비동기 배치 API (섹션 7-3, Cohere Embed v4 미지원)

### 다중 입력 처리 장점:
- **빠른 처리**: API 호출 횟수 감소 (5개 → 1개)
- **네트워크 효율**: 오버헤드 감소
- **즉시 응답**: 동기 방식으로 즉시 결과 반환

### 처리 방식:
1. **입력 그룹화**: 여러 아이템을 하나의 요청으로 묶음
2. **일괄 임베딩**: 한 번의 API 호출로 모든 임베딩 생성
3. **벌크 인덱싱**: OpenSearch bulk API로 일괄 업로드

⚠️ **참고**: 입력 그룹 크기는 API 제한(6MB)을 고려하여 조정

In [ ]:
# 다중 입력 임베딩 생성 함수
def generate_batch_embeddings(items_data):
    """
    여러 아이템의 임베딩을 한 번의 API 호출로 생성
    
    이 함수는 Cohere API의 다중 입력 기능을 활용합니다.
    Bedrock Batch Inference API와는 다른 방식입니다.
    
    Args:
        items_data: [(text, image_base64), ...] 형태의 리스트
    
    Returns:
        임베딩 벡터 리스트
    """
    # 다중 입력 페이로드 구성
    inputs = []
    for text, image_base64 in items_data:
        if image_base64:
            inputs.append({
                "content": [
                    {"type": "text", "text": text},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}}
                ]
            })
        else:
            inputs.append({
                "content": [{"type": "text", "text": text}]
            })
    
    payload = {
        "input_type": "search_document",
        "embedding_types": ["float"],
        "inputs": inputs
    }
    
    # API 호출
    response = bedrock_client.invoke_model(
        modelId='cohere.embed-v4:0',
        body=json.dumps(payload),
        contentType='application/json'
    )
    
    result = json.loads(response['body'].read())
    
    # 임베딩 추출
    if isinstance(result['embeddings'], list):
        return [emb["float"] for emb in result['embeddings']]
    else:
        return result['embeddings']['float']

print("✅ 다중 입력 임베딩 함수 정의 완료")

In [ ]:
# 다중 입력 처리로 임베딩 생성 및 업로드
import time

batch_processed_items = []
group_size = 5  # 한 번의 API 호출에 포함할 아이템 수 (필요시 조정)

print(f"🚀 다중 입력 처리 시작 (그룹 크기: {group_size})\n")
start_time = time.time()

# 그룹 단위로 처리
for i in range(0, len(sample_items), group_size):
    group = sample_items[i:i+group_size]
    group_num = i // group_size + 1
    
    print(f"📦 그룹 {group_num} 처리 중 ({len(group)}개 아이템)...")
    
    # 입력 데이터 준비
    batch_data = []
    for item in group:
        text = f"Title: {item['title']}\nDescription: {item['description']}\nCategory: {item['category']}\nBrand: {item['brand']}\nColor: {item['color']}\nPrice: ${item['price']}"
        image_path = f"data/images/{item['item_id']}.jpg"
        image_base64 = encode_image(image_path)
        batch_data.append((text, image_base64))
    
    # 다중 입력으로 임베딩 생성 (1회 API 호출)
    embeddings = generate_batch_embeddings(batch_data)
    
    # OpenSearch bulk 작업 준비
    bulk_body = []
    for idx, (item, embedding) in enumerate(zip(group, embeddings)):
        # 인덱스 액션
        bulk_body.append({"index": {"_index": INDEX_NAME}})
        
        # 문서 데이터
        doc = {
            **item,
            'multimodal_embedding': embedding,
            'created_at': datetime.now().isoformat()
        }
        bulk_body.append(doc)
        batch_processed_items.append(doc)
    
    # Bulk 업로드 (타임아웃 5분)
    response = client.bulk(body=bulk_body, request_timeout=300)
    
    if response['errors']:
        print(f"   ⚠️ 일부 문서 업로드 실패")
        for item in response['items']:
            if 'error' in item['index']:
                print(f"      - 오류: {item['index']['error']}")
    else:
        print(f"   ✅ 그룹 {group_num} 완료 ({len(group)}개 아이템 업로드)")
    
    # API 호출 간 짧은 대기 (선택사항)
    if i + group_size < len(sample_items):
        time.sleep(0.5)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\n✨ 다중 입력 처리 완료!")
print(f"📊 처리된 아이템: {len(batch_processed_items)}개")
print(f"⏱️ 소요 시간: {elapsed_time:.2f}초")
print(f"⚡ 평균 처리 속도: {len(batch_processed_items)/elapsed_time:.2f}개/초")

# 기존 방식과 비교
sequential_time = len(sample_items) * 2  # 2초씩 대기
print(f"\n💡 성능 비교:")
print(f"   - 순차 처리 (1개씩) 예상 시간: ~{sequential_time}초")
print(f"   - 다중 입력 처리 ({group_size}개씩) 실제 시간: {elapsed_time:.2f}초")
print(f"   - 시간 절약: {sequential_time - elapsed_time:.2f}초 ({(1 - elapsed_time/sequential_time)*100:.1f}% 단축)")

## 🚀 7-2-1. 대용량 데이터 최적화 (옵션 1: 큰 그룹 크기)

만 개 이상의 대량 데이터를 처리할 때 한 번의 API 호출에 포함할 입력 수를 늘려 처리 시간을 단축합니다.

### 📝 처리 방식:
- 한 번의 `invoke_model` 호출에 여러 입력을 포함
- Bedrock Batch Inference API와는 다른 방식

### 성능 예측 (10,000개 기준):
- **그룹 크기 20**: ~16분 (500회 API 호출)
- **그룹 크기 50**: ~10분 (200회 API 호출) ⭐ 권장
- **그룹 크기 100**: ~6-7분 (100회 API 호출)

### 주의사항:
- 그룹 크기가 클수록 메모리 사용량 증가
- API 요청 크기 제한(6MB) 고려
- 이미지 크기에 따라 그룹 크기 조정 필요

In [ ]:
# 대용량 데이터 처리 설정
LARGE_BATCH_SIZE = 50  # 대용량 처리용 배치 크기

# 예시: 샘플 데이터를 복제하여 대용량 시뮬레이션 (실제로는 실제 데이터 사용)
# large_dataset = sample_items * 2000  # 10,000개 시뮬레이션
large_dataset = sample_items  # 실제 사용 시 실제 데이터로 교체

print(f"📊 대용량 배치 처리 설정")
print(f"   - 데이터 수: {len(large_dataset)}개")
print(f"   - 배치 크기: {LARGE_BATCH_SIZE}개")
print(f"   - 예상 배치 수: {(len(large_dataset) + LARGE_BATCH_SIZE - 1) // LARGE_BATCH_SIZE}개")
print(f"   - 예상 소요 시간: ~{((len(large_dataset) + LARGE_BATCH_SIZE - 1) // LARGE_BATCH_SIZE) * 3 / 60:.1f}분")

In [ ]:
# 진행 상황 모니터링을 위한 tqdm 설치 (필요시)
try:
    from tqdm import tqdm
except ImportError:
    print("📦 tqdm 설치 중...")
    !pip install tqdm -q
    from tqdm import tqdm

print("✅ 진행 상황 모니터링 준비 완료")

In [ ]:
# 옵션 1: 큰 배치 크기로 처리 (진행 상황 표시)
import time
from tqdm import tqdm

large_batch_items = []
failed_items = []

print(f"🚀 대용량 배치 처리 시작 (배치 크기: {LARGE_BATCH_SIZE})\n")
start_time = time.time()

# 진행 상황 표시와 함께 배치 처리
total_batches = (len(large_dataset) + LARGE_BATCH_SIZE - 1) // LARGE_BATCH_SIZE

for i in tqdm(range(0, len(large_dataset), LARGE_BATCH_SIZE), 
              total=total_batches,
              desc="배치 처리",
              unit="batch"):
    
    batch = large_dataset[i:i+LARGE_BATCH_SIZE]
    
    try:
        # 배치 데이터 준비
        batch_data = []
        for item in batch:
            text = f"Title: {item['title']}\nDescription: {item['description']}\nCategory: {item['category']}\nBrand: {item['brand']}\nColor: {item['color']}\nPrice: ${item['price']}"
            image_path = f"data/images/{item['item_id']}.jpg"
            image_base64 = encode_image(image_path)
            batch_data.append((text, image_base64))
        
        # 배치 임베딩 생성
        embeddings = generate_batch_embeddings(batch_data)
        
        # OpenSearch bulk 작업 준비
        bulk_body = []
        for item, embedding in zip(batch, embeddings):
            bulk_body.append({"index": {"_index": INDEX_NAME}})
            doc = {
                **item,
                'multimodal_embedding': embedding,
                'created_at': datetime.now().isoformat()
            }
            bulk_body.append(doc)
            large_batch_items.append(doc)
        
        # Bulk 업로드 (타임아웃 5분)
        response = client.bulk(body=bulk_body, request_timeout=300)
        
        if response['errors']:
            for item in response['items']:
                if 'error' in item['index']:
                    failed_items.append(item)
    
    except Exception as e:
        tqdm.write(f"⚠️ 배치 {i//LARGE_BATCH_SIZE + 1} 처리 실패: {e}")
        failed_items.extend(batch)
        # 에러 발생 시 잠시 대기 후 계속
        time.sleep(2)
    
    # API 제한 방지를 위한 짧은 대기
    if i + LARGE_BATCH_SIZE < len(large_dataset):
        time.sleep(0.3)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\n✨ 대용량 배치 처리 완료!")
print(f"📊 처리 결과:")
print(f"   - 성공: {len(large_batch_items)}개")
print(f"   - 실패: {len(failed_items)}개")
print(f"   - 성공률: {len(large_batch_items)/len(large_dataset)*100:.1f}%")
print(f"⏱️ 소요 시간: {elapsed_time:.2f}초 ({elapsed_time/60:.1f}분)")
print(f"⚡ 평균 처리 속도: {len(large_batch_items)/elapsed_time:.2f}개/초")

if failed_items:
    print(f"\n⚠️ 실패한 아이템이 있습니다. 재시도를 권장합니다.")

## ⚡ 7-2-2. 병렬 다중 입력 처리 (옵션 2: 멀티스레드)

여러 API 호출을 동시에 실행하여 처리 시간을 더욱 단축합니다.

### 📝 처리 방식:
- 멀티스레드로 여러 `invoke_model` 호출을 병렬 실행
- 각 호출은 여러 입력을 포함 (다중 입력)
- Bedrock Batch Inference API와는 다른 방식

### 성능 예측 (10,000개 기준):
- **단일 스레드**: ~10분 (200회 API 호출)
- **3개 병렬**: ~3-4분 (200회 API 호출을 3개 스레드로 분산) ⭐ 권장
- **5개 병렬**: ~2-3분 (API 제한 주의)

### 주의사항:
- 병렬 처리 수가 많을수록 API 제한에 걸릴 위험 증가
- 3-5개 병렬 처리 권장
- 메모리 사용량 증가

In [ ]:
# 병렬 처리 함수 정의
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from threading import Lock

# 스레드 안전한 카운터
progress_lock = Lock()
processed_count = 0

def process_single_batch(batch, batch_id):
    """
    단일 배치를 처리하는 함수
    """
    global processed_count
    
    try:
        # 배치 데이터 준비
        batch_data = []
        for item in batch:
            text = f"Title: {item['title']}\nDescription: {item['description']}\nCategory: {item['category']}\nBrand: {item['brand']}\nColor: {item['color']}\nPrice: ${item['price']}"
            image_path = f"data/images/{item['item_id']}.jpg"
            image_base64 = encode_image(image_path)
            batch_data.append((text, image_base64))
        
        # 배치 임베딩 생성
        embeddings = generate_batch_embeddings(batch_data)
        
        # OpenSearch bulk 작업 준비
        bulk_body = []
        docs = []
        for item, embedding in zip(batch, embeddings):
            bulk_body.append({"index": {"_index": INDEX_NAME}})
            doc = {
                **item,
                'multimodal_embedding': embedding,
                'created_at': datetime.now().isoformat()
            }
            bulk_body.append(doc)
            docs.append(doc)
        
        # Bulk 업로드 (타임아웃 5분)
        response = client.bulk(body=bulk_body, request_timeout=300)
        
        # 진행 상황 업데이트
        with progress_lock:
            processed_count += len(batch)
        
        return {
            'success': True,
            'batch_id': batch_id,
            'count': len(batch),
            'docs': docs,
            'errors': response.get('errors', False)
        }
    
    except Exception as e:
        return {
            'success': False,
            'batch_id': batch_id,
            'count': len(batch),
            'error': str(e)
        }

print("✅ 병렬 처리 함수 정의 완료")

In [ ]:
# 옵션 2: 병렬 배치 처리
from tqdm import tqdm

PARALLEL_BATCH_SIZE = 50
MAX_WORKERS = 3  # 동시 처리 스레드 수 (3-5 권장)

parallel_batch_items = []
parallel_failed_items = []
processed_count = 0

print(f"🚀 병렬 배치 처리 시작")
print(f"   - 배치 크기: {PARALLEL_BATCH_SIZE}개")
print(f"   - 병렬 처리 수: {MAX_WORKERS}개")
print(f"   - 데이터 수: {len(large_dataset)}개\n")

start_time = time.time()

# 배치 그룹 생성
batches = []
for i in range(0, len(large_dataset), PARALLEL_BATCH_SIZE):
    batch = large_dataset[i:i+PARALLEL_BATCH_SIZE]
    batches.append((batch, i // PARALLEL_BATCH_SIZE))

# 병렬 처리 실행
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # 모든 배치 작업 제출
    futures = {executor.submit(process_single_batch, batch, batch_id): batch_id 
               for batch, batch_id in batches}
    
    # 진행 상황 표시
    with tqdm(total=len(batches), desc="병렬 처리", unit="batch") as pbar:
        for future in as_completed(futures):
            result = future.result()
            
            if result['success']:
                parallel_batch_items.extend(result['docs'])
                if result['errors']:
                    tqdm.write(f"⚠️ 배치 {result['batch_id']}: 일부 문서 업로드 실패")
            else:
                tqdm.write(f"❌ 배치 {result['batch_id']} 실패: {result['error']}")
                parallel_failed_items.append(result)
            
            pbar.update(1)
            pbar.set_postfix({'성공': len(parallel_batch_items), '실패': len(parallel_failed_items)})

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\n✨ 병렬 배치 처리 완료!")
print(f"📊 처리 결과:")
print(f"   - 성공: {len(parallel_batch_items)}개")
print(f"   - 실패: {len(parallel_failed_items)}개 배치")
print(f"   - 성공률: {len(parallel_batch_items)/len(large_dataset)*100:.1f}%")
print(f"⏱️ 소요 시간: {elapsed_time:.2f}초 ({elapsed_time/60:.1f}분)")
print(f"⚡ 평균 처리 속도: {len(parallel_batch_items)/elapsed_time:.2f}개/초")

# 단일 스레드와 비교
single_thread_time = (len(large_dataset) / PARALLEL_BATCH_SIZE) * 3
speedup = single_thread_time / elapsed_time
print(f"\n💡 성능 비교:")
print(f"   - 단일 스레드 예상: ~{single_thread_time/60:.1f}분")
print(f"   - 병렬 처리 실제: {elapsed_time/60:.1f}분")
print(f"   - 속도 향상: {speedup:.1f}배")

if parallel_failed_items:
    print(f"\n⚠️ {len(parallel_failed_items)}개 배치 실패. 재시도를 권장합니다.")

### 📊 대용량 데이터 처리 요약

**10,000개 데이터 처리 시간 비교:**

| 방식 | 소요 시간 | 처리 속도 | 난이도 |
|------|----------|----------|--------|
| 순차 처리 (섹션 7) | ~5.5시간 | 0.5개/초 | 쉬움 |
| 동기 배치 (섹션 7-2) | ~10분 | 16개/초 | 쉬움 |
| 큰 배치 (섹션 7-2-1) | ~6-10분 | 16-27개/초 | 쉬움 |
| 병렬 배치 (섹션 7-2-2) | ~3-4분 | 40-55개/초 | 중간 |

**권장 사항:**
- 📦 **< 1,000개**: 섹션 7-2 (기본 배치)
- 📦 **1,000-10,000개**: 섹션 7-2-1 (큰 배치)
- ⚡ **> 10,000개**: 섹션 7-2-2 (병렬 배치)

**주의사항:**
- 병렬 처리 시 API 제한에 주의
- 메모리 사용량 모니터링
- 실패한 배치는 재시도 로직 구현 권장

## 🔍 8. 멀티모달 검색 함수 정의

다양한 검색 모드를 지원하는 통합 검색 함수를 구현합니다.

### 검색 모드:
- **텍스트 검색**: 텍스트 쿼리만 사용
- **이미지 검색**: 이미지 쿼리만 사용  
- **멀티모달 검색**: 텍스트 + 이미지 결합 쿼리

### 검색 과정:
1. **쿼리 임베딩**: 입력을 1536차원 벡터로 변환
2. **k-NN 검색**: HNSW 인덱스에서 유사 벡터 탐색
3. **결과 반환**: 유사도 점수와 함께 상위 k개 결과

In [ ]:
def search(query_text=None, query_image_path=None, top_k=3):
    # 쿼리 임베딩 생성
    if query_image_path and query_text:
        # 멀티모달 검색 (텍스트 + 이미지)
        image_base64 = encode_image(query_image_path)
        payload = {
            "input_type": "search_query",
            "embedding_types": ["float"],
            "inputs": [{
                "content": [
                    {"type": "text", "text": query_text},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}}
                ]
            }]
        }
    elif query_image_path:
        # 이미지 전용 검색
        image_base64 = encode_image(query_image_path)
        payload = {
            "input_type": "search_query",
            "embedding_types": ["float"],
            "inputs": [{
                "content": [
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}}
                ]
            }]
        }
    else:
        # 텍스트 전용 검색
        payload = {
            "input_type": "search_query",
            "embedding_types": ["float"],
            "texts": [query_text]
        }
    
    response = bedrock_client.invoke_model(
        modelId='cohere.embed-v4:0',
        body=json.dumps(payload),
        contentType='application/json'
    )
    
    result = json.loads(response['body'].read())
    if isinstance(result['embeddings'], list):
        query_embedding = result['embeddings'][0]["float"]
    else:
        query_embedding = result['embeddings']['float'][0]
    
    # 검색 실행 (타임아웃 5분)
    search_query = {
        "size": top_k,
        "query": {
            "knn": {
                "multimodal_embedding": {
                    "vector": query_embedding,
                    "k": top_k
                }
            }
        },
        "_source": {"excludes": ["multimodal_embedding"]}
    }
    
    response = client.search(index=INDEX_NAME, body=search_query, request_timeout=300)
    
    results = []
    for hit in response['hits']['hits']:
        result = hit['_source']
        result['score'] = hit['_score']
        results.append(result)
    
    return results

In [ ]:
def display_results(results, query_desc):
    print(f"\n=== 검색 결과: {query_desc} ===")
    
    if not results:
        print("❌ 검색 결과가 없습니다.")
        return
    
    print(f"📊 총 {len(results)}개 결과 발견\n")
    
    for i, item in enumerate(results, 1):
        print(f"{i}. {item['title']} (점수: {item['score']:.4f})")
        print(f"   {item['category']} | {item['brand']} | {item['color']} | ${item['price']}")
        print(f"   {item['description']}")
        print()

print("🔍 검색 함수 정의 완료")

## 🧪 9. 검색 기능 테스트

구현된 멀티모달 검색 시스템의 다양한 검색 모드를 테스트합니다.

### 테스트 시나리오:
1. **텍스트 검색**: "블루 캐주얼 재킷" 키워드 검색
2. **이미지 검색**: 특정 이미지와 유사한 아이템 검색
3. **멀티모달 검색**: 텍스트 설명 + 이미지 결합 검색

각 검색 결과는 유사도 점수와 함께 표시됩니다.

In [ ]:
# 1. 텍스트 검색
results = search(query_text="블루 캐주얼 재킷 일상복")
display_results(results, "텍스트 검색: 블루 캐주얼 재킷")

# 2. 이미지 검색
results = search(query_image_path="data/images/item_001.jpg")
display_results(results, "이미지 검색: item_001.jpg")

# 3. 멀티모달 검색
results = search(
    query_text="우아한 정장 특별한 행사용",
    query_image_path="data/images/item_002.jpg"
)
display_results(results, "멀티모달 검색: 우아한 정장 + 이미지")


In [ ]:
# 인덱스 존재 및 문서 수 확인
print(f"인덱스 존재: {client.indices.exists(index=INDEX_NAME, request_timeout=300)}")

# 문서 수 확인
response = client.search(index=INDEX_NAME, body={"query": {"match_all": {}}, "size": 0}, request_timeout=300)
doc_count = response['hits']['total']['value']
print(f"문서 수: {doc_count}")

if doc_count == 0:
    print("❌ 문서가 없습니다. 임베딩 업로드가 실패했습니다.")
    print("API 제한이 해제된 후 임베딩 생성 셀을 다시 실행하세요.")

## 📊 11. 솔루션 요약

구축된 멀티모달 검색 시스템의 주요 정보를 요약합니다.

### 🎉 구축 완료된 기능:
- ✅ **HNSW 벡터 인덱스**: 고성능 실시간 검색
- ✅ **멀티모달 임베딩**: 텍스트 + 이미지 통합
- ✅ **확장 가능한 아키텍처**: OpenSearch Serverless
- ✅ **유연한 검색**: 3가지 검색 모드 지원

### 🚀 성능 특징:
- **검색 속도**: O(log N) HNSW 알고리즘
- **임베딩 품질**: Cohere Embed v4 (1536차원)
- **유사도 측정**: 코사인 유사도
- **자동 스케일링**: OpenSearch Serverless

In [ ]:
print("=== 멀티모달 임베딩 솔루션 정보 ===")
print(f"OpenSearch 엔드포인트: {ENDPOINT}")
print(f"인덱스: {INDEX_NAME}")
print(f"임베딩 모델: Cohere embed-v4")
print(f"임베딩 차원: 1536")
print(f"처리된 아이템: {len(processed_items)}개")
print("\n=== 솔루션 구축 완료 ===")